<a href="https://colab.research.google.com/github/WoradeeKongthong/association_rule_learning/blob/master/03_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Dataset
data source :  
https://www.kaggle.com/rounakbanik/the-movies-dataset?select=ratings_small.csv  
https://www.kaggle.com/rounakbanik/the-movies-dataset?select=movies_metadata.csv

In [0]:
movies = pd.read_csv('movies_metadata.csv',usecols=[5,8],dtype={'id':str},index_col='id')
ratings = pd.read_csv('ratings_small.csv',dtype={'movieId':str},usecols=[0,1,2])

In [257]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45466 entries, 862 to 461257
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   original_title  45466 non-null  object
dtypes: object(1)
memory usage: 710.4+ KB


In [258]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100004 non-null  int64  
 1   movieId  100004 non-null  object 
 2   rating   100004 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.3+ MB


In [0]:
df = ratings.join(movies, on='movieId', how='left')

In [260]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100009 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   userId          100009 non-null  int64  
 1   movieId         100009 non-null  object 
 2   rating          100009 non-null  float64
 3   original_title  44994 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 3.8+ MB


In [0]:
df.dropna(axis=0, inplace=True)

In [262]:
df

,userId,movieId,rating,original_title
10,1,1371,2.5,Rocky III
11,1,1405,1.0,Greed
13,1,2105,4.0,American Pie
15,1,2193,2.0,My Tutor
16,1,2294,2.0,Jay and Silent Bob Strike Back
...,...,...,...,...
99983,671,4995,4.0,Boogie Nights
99992,671,5816,4.0,Ober
99993,671,5902,3.5,A Bridge Too Far
99996,671,5991,4.5,Der letzte Mann


In [263]:
df['rating'].value_counts()

4.0    13163
3.0     9899
5.0     7335
3.5     4038
2.0     3237
4.5     3017
2.5     1726
1.0     1518
1.5      652
0.5      409
Name: rating, dtype: int64

I consider the movies that the user likes,  
so I'll choose the movies that get the rating more than 2.5 (maximum rating = 5)

In [0]:
movie_df = df[df['rating']>2.5][['userId','original_title']].copy()

In [265]:
movie_df

,userId,original_title
13,1,American Pie
21,2,The Dark
26,2,2001: A Space Odyssey
27,2,Trois couleurs : Rouge
28,2,Der Himmel über Berlin
...,...,...
99983,671,Boogie Nights
99992,671,Ober
99993,671,A Bridge Too Far
99996,671,Der letzte Mann


In [266]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37452 entries, 13 to 99997
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userId          37452 non-null  int64 
 1   original_title  37452 non-null  object
dtypes: int64(1), object(1)
memory usage: 877.8+ KB


Create list-of-list input for Apriori  

In [0]:
users = [movie_df['userId'].min()]
basket = []
watch_lists = []

for i in range(len(movie_df)):
  user_id = movie_df.iloc[i,0]
  movie = movie_df.iloc[i,1]

  if user_id in users :
    # put this user' movie into her basket
    basket.append(movie)
  else :
    
    # I will choose only the user who watch the movie I'm interested in (movie_title)
    #if movie_title in basket :
      # put the former user's basket into watch_lists
    watch_lists.append(basket)
    
    # empty basket
    basket = []

    # add new user_id into users
    users.append(user_id)
    # put this new user's movie into basket
    basket.append(movie)

#if movie_title in basket :
watch_lists.append(basket)


In [268]:
len(watch_lists)

671

In [269]:
watch_lists[:2]

[['American Pie'],
 ['The Dark',
  '2001: A Space Odyssey',
  'Trois couleurs : Rouge',
  'Der Himmel über Berlin',
  '48 Hrs.',
  'Lost in Translation',
  "Ocean's Eleven",
  'Back to the Future Part II',
  'Star Trek IV: The Voyage Home',
  'A Clockwork Orange',
  'Lucky Number Slevin',
  'Berlin: Die Sinfonie der Grosstadt',
  'Czlowiek z zelaza',
  'Stand by Me',
  'Pocketful of Miracles',
  'Live and Let Die',
  'Cat on a Hot Tin Roof',
  'Lili Marleen',
  'Le Mépris',
  'Batman Begins',
  'Das weisse Rauschen',
  "Dave Chappelle's Block Party",
  'Terminator 3: Rise of the Machines',
  'La science des rêves',
  'Catwoman',
  'Night on Earth',
  'Crustacés et coquillages',
  'The Devil Wears Prada',
  'Batman Returns',
  'A Nightmare on Elm Street',
  'Sommer vorm Balkon',
  'Romeo + Juliet',
  'Sissi',
  'My Own Private Idaho',
  'Monsoon Wedding',
  'The Green Mile',
  'Reservoir Dogs',
  'Love Actually',
  'Notting Hill',
  'Once Were Warriors',
  'Je ne suis pas là pour être a

# Priori model

In [270]:
! pip install apyori

In [0]:
from apyori import apriori

rules = apriori(watch_lists, min_support = 0.1, min_confidence = 0.5 ,min_lift = 3, max_length =2)

# Result

In [0]:
result = list(rules)

In [276]:
len(result)

16

In [284]:
result[1]

RelationRecord(items=frozenset({'Bad Boys II', 'And Then There Were None'}), support=0.11773472429210134, ordered_statistics=[OrderedStatistic(items_base=frozenset({'And Then There Were None'}), items_add=frozenset({'Bad Boys II'}), confidence=0.6422764227642276, lift=3.6522667769050567), OrderedStatistic(items_base=frozenset({'Bad Boys II'}), items_add=frozenset({'And Then There Were None'}), confidence=0.6694915254237288, lift=3.652266776905057)])

In [0]:
results1 = []
for i in range(len(result)):
  item1 = list(result[i].ordered_statistics[0][0])[0]
  item2 = list(result[i].ordered_statistics[0][1])[0]
  support = result[i].support
  confidence = result[i].ordered_statistics[0][2]
  lift = result[i].ordered_statistics[0][3]

  row = (item1, item2, support, confidence, lift)
  results1.append(row)

labels = ['movieA','movieB','support','confidence','lift']
result_df = pd.DataFrame(results1, columns=labels)

In [289]:
result_df.sort_values('lift', ascending=False)

,movieA,movieB,support,confidence,lift
5,Big Fish,Psycho,0.107303,0.727273,4.518519
3,Bang Boom Bang - Ein todsicheres Ding,Syriana,0.111773,0.688073,3.946130
1,And Then There Were None,Bad Boys II,0.117735,0.642276,3.652267
10,Knallhart,Rain Man,0.105812,0.887500,3.544717
14,Psycho,Reservoir Dogs,0.107303,0.666667,3.441026
2,Back to the Future Part II,Crustacés et coquillages,0.105812,0.563492,3.437302
13,Point Break,The Talented Mr. Ripley,0.116244,0.634146,3.350490
11,Lost in Translation,Rain Man,0.102832,0.821429,3.280825
6,Big Fish,Titanic,0.110283,0.747475,3.278141
12,Lost in Translation,The Conversation,0.105812,0.845238,3.259510


Let's discuss about the result.  



*   Rule 1 : Big Fish & Psycho  
    - 10.73% of the users watch both of these movies  
    - the confidence that the users who watch Big Fish will watch Psycho is 72.73%  
    - the likelihood that the users will watch Big Fish and Psycho together is 4.52 times more than the likelihood that the users watch only just Psycho.  
